In [84]:
pID = 3
pID = 'sub-0' + "%02d" % (pID)

In [85]:
import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, KFold

import pickle, json
import scipy.io
from bci_funcs import windowed_mean, base_correct

#path = '/Users/lukasgehrke/Documents/publications/2021-fastReach/data/study/eeglab2python/'
# path = 'P:\\Lukas_Gehrke\\2021-fastReach\\data\\study\\eeglab2python\\'
path = '/Volumes/projects/Lukas_Gehrke/2021-fastReach/data/study/eeglab2python/'

# Preconscious Augmentation

This script trains the classifier for a brain-computer interface that controls electrical muscle stimulation in the preconscious augmentation experiment.
The functions used to build the feature vectors are the same that are used for the online application and are found in 'bci_funcs'

A two class linear discriminant model is fitted to idle and pre-movement EEG training data. The model and a channel selection is saved.

In [86]:
pre_move = scipy.io.loadmat(path+pID+'/pre_move_Baseline.mat')
idle = scipy.io.loadmat(path+pID+'/idle_Baseline.mat')
chans = np.array(pd.read_csv(path+pID+'/sel_chans.csv', header=None)).flatten()
chans = chans - 1 # Matlab to python indexing

# chans = chans[0:4]
chans = chans.tolist() # first three channels are C3, C4, Cz, in that order. Might ignore, then start index at 3

data_srate = 250
windows = 10
baseline = data_srate/windows

idle = idle['idle'][chans,:,:]
pre_move = pre_move['pre_move'][chans,:,:]

idle_means = np.zeros((idle.shape[2], idle.shape[0] * (windows)))
pre_move_means = np.zeros((pre_move.shape[2], pre_move.shape[0] * (windows)))

target_class = 1
threshold = 1
classifier_update_rate = 10 # samples

bci_params = dict(((k, eval(k)) for k in ('chans', 'windows', 'baseline_ix', 'baseline', 'target_class', 'threshold', 'data_srate', 'classifier_update_rate')))
with open(path+pID+'/bci_params.json', 'w') as f:
    json.dump(bci_params, f)

for trial_ix in range(0, pre_move.shape[2]):
    tmp = base_correct(pre_move[:,:,trial_ix], baseline-1)
    pre_move_means[trial_ix, :] = windowed_mean(tmp, windows = windows).flatten()

for trial_ix in range(0, idle.shape[2]):
    tmp = base_correct(idle[:,:,trial_ix], baseline-1)
    idle_means[trial_ix, :] = windowed_mean(tmp, windows = windows).flatten()

data = np.concatenate((pre_move_means, idle_means), axis = 0)

In [87]:
pre_move_class = np.ones((pre_move_means.shape[0], 1))
idle_class = np.zeros((idle_means.shape[0], 1))
classes = np.concatenate((pre_move_class, idle_class)).ravel()

In [88]:
clf = LDA(solver='eigen', shrinkage='auto')
clf.fit(data, classes)

kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
cv_results = cross_val_score(clf, data, classes, cv=kfolds)
print(cv_results.mean())
print(cv_results.std())

filename = path+pID+'/model_'+pID+'_eeg.sav'
pickle.dump(clf, open(filename, 'wb'))

0.6857142857142857
0.03499271061118824
